In [22]:
import ast
import datetime
import hashlib
import json
import logging
import math
import re
import time
import uuid
import urllib3

import redis
import requests
import xmltodict
from pymongo import MongoClient
from bs4 import BeautifulSoup
from connection import get_in,ConnectionTask
from pprint import pprint
import conf

c=ConnectionTask()
urllib3.disable_warnings()

In [2]:
def ctrip_fetch(data):
    """酒店订单精简信息查询接口 D_HotelOrderMiniInfo
    """
    app_code = data['appCode'] if data.get('appCode') == 'oklife' else 'other'
    ping_url = 'http://{}/Hotel/D_HotelOrderMiniInfo.asmx'.format(conf.CTRIP_API_URL)
    ts = int(time.time())
    secret = hashlib.md5(conf.CTRIP_KEY[app_code].encode('utf8')).hexdigest().upper()
    sign = hashlib.md5('{0}{1}{2}{3} D_HotelOrderMiniInfo'.format(
        ts, conf.CTRIP_AID[app_code], secret, conf.CTRIP_SID[app_code]).encode('utf-8')).hexdigest().upper()
    request_xml = conf.CTRIP_ORDER_XML.format(conf.CTRIP_AID[app_code], conf.CTRIP_SID[app_code], ts, sign, data['partnerId'])
    encode_xml = request_xml.encode('utf-8')
    headers = {'Content-Type': 'text/xml; charset=utf-8',
               'Host': 'openapi.ctrip.com', 'Content-Length': str(len(encode_xml)),
               'SOAPAction': "http://ctrip.com/Request"}
    resp = requests.post(url=ping_url, data=encode_xml, headers=headers).text
    xmlt_data = xmltodict.parse(xmltodict.parse(resp)['soap:Envelope']['soap:Body']['RequestResponse']['RequestResult'])
    xmlt_data = (json.loads((json.dumps(xmlt_data, ensure_ascii=False, indent=2)).replace('@', '')))
    return xmlt_data
    resp_data = xmlt_data['Response']['HotelOrderMiniInfoResponse']
    return resp_data

In [81]:
data={'appCode': 'blm', 'partnerId': '6278772602', 'orderId': 'hnx004401624647982'}
ctrip_fetch(data)

{'Response': {'Header': {'AccessCount': '5000',
   'CurrentCount': '1',
   'RecentlyTime': '2018-05-08 17:53:09',
   'ReferenceID': '19b2cbae-bc4d-4338-95f3-37f2fe5b6ae6',
   'ResetTime': '2018-05-08 17:54:09',
   'ResultCode': 'Success',
   'ShouldRecordPerformanceTime': 'False',
   'Timestamp': '2018-05-08T17:53:09.1895656+08:00'},
  'HotelOrderMiniInfoResponse': {'AllianceId': '328727',
   'CheckInDate': '2018-05-10T00:00:00+08:00',
   'CheckOutDate': '2018-05-11T00:00:00+08:00',
   'FromOrderId': '0',
   'HotelId': '970587',
   'HotelName': '全季酒店(天津友谊路店)',
   'OrderDate': '2018-05-08T17:18:38+08:00',
   'OrderId': '6278772602',
   'OrderStatus': '已取消',
   'Ouid': None,
   'Price': '270',
   'Quantity': '1',
   'Sid': '810129'}}}

In [ ]:
def token_fetch_and_update():
    """获取、更新携程access token,存储于radis中,失效后可再次获取
    """
    token_payload = {
        'AID': conf.XIECHENG_AID,
        'SID': conf.XIECHENG_SID,
        'KEY': conf.XIECHENG_TOKEN_KEY,
    }
    token = 'https://openserviceauth.ctrip.com/openserviceauth/authorize.ashx'
    token_result = requests.post(token, params=token_payload)
    Refresh_Token = token_result.json()['Refresh_Token']

    atoken_payload = {
        'AID': conf.XIECHENG_AID,
        'SID': conf.XIECHENG_SID,
        'refresh_token': Refresh_Token,
    }
    Atoken = 'https://openserviceauth.ctrip.com/openserviceauth/refresh.ashx'

    Atoken_result = requests.post(Atoken, params=atoken_payload)
    Access_Token = Atoken_result.json()['Access_Token']
    # 存贮token与过期时间字段
    c.dpprdb(0).hset('ctrip_token', 'access_token', Access_Token)
    c.dpprdb(0).hset('ctrip_token', 'expire_time', datetime.datetime.now() + datetime.timedelta(seconds=60 * 9))
    return ['xiecheng_access_token update is OK!']


In [ ]:
def city_get_full_hotel_ids(city, page_index=1):
    """根据城市ID，查询酒店全量ID列表，并调用静态信息接口
    """
    Access_Token = c.dpprdb(0).hget('ctrip_token', 'access_token')
    print(Access_Token)
    # 从redis获取Access token
    payload = {
        'AID': conf.XIECHENG_AID,
        'SID': conf.XIECHENG_SID,
        'ICODE': conf.XIECHENG_ICODE,
        'UUID': conf.XIECHENG_UUID,
        'Token': Access_Token,
        'mode': 1,
        'format': 'json'
    }
    url_html = 'http://openservice.ctrip.com/OpenService/ServiceProxy.ashx'
    data = {
        "City": city,
        "PageIndex": str(page_index),
        "PageSize": "5000"
    }

    try:
        requests_result = requests.post(url_html, params=payload, data=data)
        resp_data = requests_result.json()
        return resp_data
        total = resp_data['Total']
        if int(total) == 0:
            return ['city', city, 'total 0']
        hotel_ids = resp_data['HotelIDs'].split(',')
        return hotel_ids
    except KeyError as e:
        # 如果报错，token过期，执行token_fetch函数更新token
        print(resp_data)
        if str(datetime.datetime.now()) > (c.dpprdb(0).hget('ctrip_token', 'expire_time')):
            token_fetch_and_update()
        # 接口问题，创建一个60s延迟的自身调用异步任务
#         city_get_full_hotel_ids.apply_async(args=[city, page_index], countdown=60, queue=conf.MQ_XIECHENG)
        return['city', city, 'EOFError and Retry']

#     if page_index * 5000 < total:
        # 执行下一页的参数查询
#         city_get_full_hotel_ids.apply_async(args=[city, page_index + 1], queue=conf.MQ_XIECHENG)
    for hotel_ids_index in range(len(hotel_ids) // 10 + 1):
        return hotel_ids[hotel_ids_index * 10:(hotel_ids_index + 1) * 10]
        # 切片酒店ID列表，调用静态信息接口
#         hotel_ids_message_fetch(args=[hotel_ids[hotel_ids_index * 10:(hotel_ids_index + 1) * 10]],
#                                             queue=conf.MQ_XIECHENG)  # async
        break  # test
    return [city, page_index, len(hotel_ids), 'is over']

In [ ]:
hotel_ids = city_get_full_hotel_ids(1236)['HotelIDs'].split(',')

In [ ]:
hotel_ids_message_fetch(['6887689', '6887690', '6887691', '6887692', '6887693', '6887695', '6887697', '6887698', '6887700', '6887701'])

In [71]:
def hotel_ids_message_fetch(hotel_ids):
    """根据酒店ID，查询酒店静态信息,并返回数据进行清洗，最后调用价格计划接口(if rateplans is True)"""
    ping_url = 'http://{}/Hotel/OTA_HotelDescriptiveInfo.asmx'.format(
        conf.XIECHENG_API_URL)
    ts = int(time.time())
    secret = hashlib.md5(conf.XIECHENG_KEY.encode('utf8')).hexdigest().upper()
    sign = hashlib.md5('{0}{1}{2}{3}OTA_Ping'.format(ts,
                                                     conf.XIECHENG_AID,
                                                     secret,
                                                     conf.XIECHENG_SID).encode('utf-8')).hexdigest().upper()
    request_xml = (conf.XIECHENG_MESSAGE_XML)
    hotel_desc = (conf.XIECHENG_MESSAGE_DESC)
    hotel_index = ''
    # 动态创建n个id的request body
    n = len(hotel_ids)
    if n == 0:
        return ['hotel_ids is []']
    for i in range(n):
        hotel_index += hotel_desc.replace('index', str(i))
    hotel_desc = hotel_index.format(hotel_ids)
    request_xml = request_xml.format(conf.XIECHENG_AID, conf.XIECHENG_SID, ts, sign, hotel_desc)
    encode_xml = request_xml.encode('utf-8')
    headers = {
        'Content-Type': 'text/xml; charset=utf-8',
        'Host': 'openapi.ctrip.com',
        'Content-Length': str(len(encode_xml))
    }

    resp = requests.post(url=ping_url, data=encode_xml, headers=headers)
    if resp.status_code != 200:
        return [resp.status_code, 'requests err', hotel_ids]
    xmlt_data = xmltodict.parse(xmltodict.parse(resp.text)['soap:Envelope']['soap:Body']['RequestResponse']['RequestResult'])
    # 调用轮子xmltodict，将xml格式数据 转换为 dict格式数据，并删除‘@’字符
    xmlt_data = (json.loads((json.dumps(xmlt_data, ensure_ascii=False, indent=2)).replace('@', '')))
#     return xmlt_data['Response']
    datas = xmlt_data['Response']['HotelResponse']['OTA_HotelDescriptiveInfoRS'][
        'HotelDescriptiveContents']['HotelDescriptiveContent']
    if not isinstance(datas, list):
        datas = [datas]
    return datas

In [77]:
resp = hotel_ids_message_fetch(['823769']) # 19:52 -21:18=8+60+18=86 /100  !! 69.7/min
json.dumps(resp,ensure_ascii=False)
# 16:25:26 -16:27:41

'[{"BrandCode": "48", "HotelCode": "823769", "HotelCityCode": "2", "HotelName": "汉庭酒店(上海松江店)", "AreaID": "125", "HotelId": "823769", "HotelInfo": {"WhenBuilt": "2008-1-1 0:00:00", "LastUpdated": "2008-01-01T00:00:00", "IsOnlineFranchise": "false", "CategoryCodes": {"SegmentCategory": {"Code": "5"}}, "Position": {"Latitude": "31.017049", "Longitude": "121.262809", "PositionTypeCode": "502"}, "Address": {"AddressLine": "荣乐东路1599号", "Zone": {"ZoneCode": "841", "ZoneName": "佘山、松江大学城"}, "CityName": "上海", "PostalCode": "201600", "TPA_Extensions": {"RoadCross": {"DescriptionText": "近松卫北路"}}}, "Services": {"Service": [{"Code": "6", "ID": "1001", "DescriptiveText": "会议厅"}, {"Code": "79", "ID": "1004", "DescriptiveText": "24小时热水"}, {"Code": "93", "ID": "1004", "DescriptiveText": "吹风机"}, {"Code": "100", "ID": "1002", "DescriptiveText": "免费停车场"}, {"Code": "183", "ID": "1004", "DescriptiveText": "电视机"}, {"Code": "219", "ID": "1004", "DescriptiveText": "手动窗帘"}, {"Code": "225", "ID": "1004", "Descrip

In [80]:
c.static_cluster_ctrip.jiudian_hotels_test.insert_many(resp)

In [79]:
c.static_cluster_ctrip.jiudian_hotels_test.find_one()

{'AffiliationInfo': {'Awards': {'Award': [{'Provider': 'HotelStarLicence',
     'Rating': '0'},
    {'Provider': 'HotelStarRate', 'Rating': '2'},
    {'Provider': 'CtripStarRate', 'Rating': '2'},
    {'Provider': 'CtripUserRate', 'Rating': '2'},
    {'Provider': 'CtripCommRate', 'Rating': '4.4'},
    {'Provider': 'CommSurroundingRate', 'Rating': '4.3'},
    {'Provider': 'CommFacilityRate', 'Rating': '4.3'},
    {'Provider': 'CommCleanRate', 'Rating': '4.6'},
    {'Provider': 'CommServiceRate', 'Rating': '4.5'}]}},
 'AreaID': '125',
 'AreaInfo': {'RefPoints': {'RefPoint': [{'CategoryCode': '1001',
     'DescriptiveText': '逸岭餐厅(宝益路)',
     'Distance': '0.001',
     'Latitude': '31.017053',
     'Longitude': '121.262826',
     'Name': '逸岭餐厅(宝益路)',
     'RefPointCategoryCode': '1',
     'RefPointName': '餐饮',
     'UnitOfMeasureCode': '2'},
    {'CategoryCode': '1001',
     'DescriptiveText': '逸岭餐厅',
     'Distance': '0.009',
     'Latitude': '31.016992',
     'Longitude': '121.262732',
   

In [ ]:
hotel_ids_rateplans_fetch(['1361135'],pay_type=1)

In [ ]:
def hotel_ids_rateplans_fetch(hotel_ids,pay_type=0):
    """ 查询酒店ID对应price日历，返回最低价格
    Args:
        pay_type(int): 0线付，1现付
    """
#     AID_FG: '263411',
#             SID_FG: '712773',
#             KEY_FG: '1F100349-B1C2-4C92-B6CD-69BE97EAF7F8',
    start = str(datetime.date.today())
    # 价格日历28天
    end = str(datetime.date.today() + datetime.timedelta(days=28))
    ping_url = 'http://{}/Hotel/OTA_HotelRatePlan.asmx'.format(conf.XIECHENG_API_URL)
    ts = int(time.time())
    secret = hashlib.md5(
        conf.XIECHENG_KEY.encode('utf8') if pay_type==0 else '1F100349-B1C2-4C92-B6CD-69BE97EAF7F8'.encode('utf8')
    ).hexdigest().upper()
    sign = hashlib.md5('{0}{1}{2}{3}OTA_HotelRatePlan'.format(ts,
                                                              conf.XIECHENG_AID if pay_type==0 else 263411,
                                                              secret,
                                                              conf.XIECHENG_SID if pay_type==0 else 712773
                                                             ).encode('utf-8')).hexdigest().upper()

    # 动态创建n个id 的request body
    request_xml = (conf.XIECHENG_PRICE_XML)
    rateplan = (conf.XIECHENG_PRICE_RATEPLAN)
    rateplans = ''
    n = len(hotel_ids)
    for index in range(n):
        rateplans += rateplan.replace('index', str(index))
    rateplans = rateplans.format(hotel_ids=hotel_ids, start=start, end=end)
    request_xml = request_xml.format(conf.XIECHENG_AID if pay_type==0 else 263411,
                                     conf.XIECHENG_SID if pay_type==0 else 712773,
                                     ts,
                                     sign,
                                     datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S.%f+08:00'),
                                     rateplans=rateplans)

    encode_xml = request_xml.encode('utf-8')
    headers = {
        'Content-Type': 'text/xml; charset=utf-8',
        'Host': 'openapi.ctrip.com',
        'Content-Length': str(len(encode_xml))
    }
    try:
        resp = requests.post(url=ping_url, data=encode_xml, headers=headers).text
        xmlt_data = xmltodict.parse(xmltodict.parse(resp)['soap:Envelope']['soap:Body']['RequestResponse']['RequestResult'])
        # xml 转 dict
        xmlt_data = (json.loads((json.dumps(xmlt_data, ensure_ascii=False, indent=2)).replace('@', '')))
#         return xmlt_data['Response']['Header']['ResultCode']
        datas = xmlt_data['Response']['HotelResponse']['OTA_HotelRatePlanRS']['RatePlans']
        return datas
    except (KeyError, Exception) as e:
        return ['err and retry,errids:', hotel_ids]
    if not isinstance(datas, list):
        datas = [datas]
    return datas
    for data in datas:
        rateplans_get_price_and_insert(data)  # pylint: disable=E1120
    return [hotel_ids, 'rateplans fetch is OK']

In [ ]:
hotel_ids_rateplans_fetch(['a82375311112223333332'])
# {'PrimaryLangID': 'zh',
#  'Success': None,
#  'TimeStamp': '2017-12-26T15:58:38.8747412+08:00',
#  'Version': '1.0',
#  'xmlns': 'http://www.opentravel.org/OTA/2003/05'}

In [ ]:
# # 携程:主动更新酒店信息
# from path import Path
# import os
# a=Path('/root/dpp/')
# a.chdir()
# from ctrip_worker import hotel_ids_message_fetch
# hotels=[]
# count=0
# for database in c.static_ctrip.collection_names()[1:]:
#     print(database)
#     if 'jiudian' not in database:
#         continue
#     for item in c.static_ctrip[database].find({'rooms.tpa_Extensions':None},{'_id':0,'id':1}):
#         hotels.append(item['id'])
#         if len(hotels)==10:
#             hotel_ids_message_fetch.apply_async(args=[hotels],queue='dpp_ctrip_tasks')
#             count+=1
#             print(count,end='\r')
#             hotels=[]
#         else:
#             continue
# if hotels:
#     print('over',hotels)
#     hotel_ids_message_fetch.apply_async(args=[hotels],queue='dpp_ctrip_tasks')

In [ ]:
from path import Path
import os
a=Path('/root/dpp/')
a.chdir()
from ctrip_worker import hotel_ids_message_fetch,jiudian_hotels_sub_library
# hotel_ids_message_fetch.apply_async(args=[['1700306']],queue='dpp_ctrip_tasks')
jiudian_hotels_sub_library.apply_async(args=['706587'],queue='dpp_ctrip_tasks')

In [ ]:
for item in c.static.jiudian_hotels.find(
    {
        'cityCode':{
            '$in':['1','2','30','32']
        }
    },
    {
        'id':1,
        '_id':0
    }
):
    jiudian_hotels_sub_library.apply_async(args=[item['id']],queue='dpp_ctrip_tasks')
    

In [ ]:
# tag(list)
rdb=c.srdb_new(0)
count=0
err_data=[]
for collection in c.dmdb_ctrip.collection_names():
    if collection in ['jiudian_hotels_sort','system.indexes']:
        continue
    for clean_datas in c.dmdb_ctrip[collection].find({},
                                                    {'_id':0,
                                                     'id':1,
                                                     'areaId':1,
                                                     'zoneCode':1,
                                                     'name':1,
                                                     'brandCode':1}):
        try:
            name=clean_datas['name']
            tag=jieba_lcut_for_search(name)
            tag.add(name)
            sentence=name
            district_name = rdb.hget('ctrip_map:district', clean_datas['areaId'])
            brand_name=rdb.hget('ctrip_map:brand',clean_datas['brandCode'])
            if not brand_name:
                continue

            if clean_datas['zoneCode']:
                zone_list = [zone for zone in rdb.hmget('ctrip_map:zone', clean_datas['zoneCode']) if zone]
                tag=tag.union(set([item for item in punctuation_replace(','.join(zone_list)).split(',') if item]))
                sentence+=' '+str(zone_list)
            if district_name:
                tag.add(district_name)
                sentence+=' '+district_name
            if brand_name:
                tag.add(brand_name)
                sentence+=' '+brand_name
            tag=tag.union(jieba_analyse_extract_tags(sentence))
            count+=1
            c.dmdb_ctrip[collection].update_one({'id':clean_datas['id']},{'$set':{'tag':list(tag)}})
            c.dmdb_ctrip['jiudian_hotels_sort'].update_one({'id':clean_datas['id']},{'$set':{'tag':list(tag)}})
#             print(collection,clean_datas['id'],brand_name,tag,count,end='\n')
            print(count,end='\r')
#             break
        except:
            err_data.append(clean_datas)
#     break

In [ ]:
c.static_ctrip['jiudian_hotels_sort'].count({'brandCode':{'$in':brand_list}})

In [ ]:
brand_list=list((rdb.hgetall('ctrip_map:brand')).keys())

In [ ]:
import jieba.analyse,jieba
import zhon.hanzi
import string

def jieba_analyse_extract_tags(sentence):
    return set(jieba.analyse.extract_tags(sentence))

def jieba_lcut_for_search(sentence):
    sentence=punctuation_replace(sentence)
    tag=set()
    resp=jieba.lcut(sentence)
    for i in range(1,len(resp)+1):
        if ',' in resp[i-1]:
            break
        tag.add(''.join(resp[:i]))
    return tag

def punctuation_replace(sentence,replace=','):
    punctuation=string.punctuation+zhon.hanzi.punctuation
    trantab=str.maketrans(punctuation, replace*len(punctuation))
    return sentence.translate(trantab).strip(replace)

In [ ]:
jieba_lcut_for_search('星程酒店(南通体育会展中心店)')

In [ ]:
# 提取ctrip各城市不同地表分类统计
ref_points,count={},0
for item in c.static.jiudian_hotels.find({},{'_id':0,'cityCode':1,'refPoints':1},no_cursor_timeout=True):
    try:
        ref_points.setdefault(item['cityCode'],{})
        count+=1
        if not item.get('refPoints'):
            continue
        if not isinstance(item['refPoints'],list):
            item['refPoints']=[item['refPoints']]
        for ref_point in item['refPoints']:
            ref_points[item['cityCode']].setdefault(ref_point['RefPointName'],[])
            ref_points[item['cityCode']].setdefault('大学',[])
            ref_points[item['cityCode']].setdefault('医院',[])
            if ref_point.get('Name'):
                if '医院' in ref_point['Name']:
                    ref_points[item['cityCode']]['医院'].append(ref_point['Name'])
                elif '大学' in ref_point['Name']:
                    ref_points[item['cityCode']]['大学'].append(ref_point['Name'])
                else:
                    ref_points[item['cityCode']][ref_point['RefPointName']].append(ref_point['Name'])
        print(f">>>cityCode:{item['cityCode']},count:{count}<<<",end='\r')
    except:
        pprint(item)

In [ ]:
city_category_name={}
for city_code,value in ref_points.items():
    city_category_name[city_code]={}
    for category,name_list in value.items():
        city_category_name[city_code][category]=[item[0] for item in Counter(name_list).most_common(20)]
#     pprint(city_category_name)

In [ ]:
with open('ctrip_refPoints.json','w') as f:
    f.write(json.dumps(city_category_name,indent=2,ensure_ascii=False))

In [ ]:
city_category_name

In [ ]:
from collections import Counter
a=Counter([1,2,3,2,3,4,5,6,6,3,2,413,13,2])
# a.update([2])
[item[0] for item in a.most_common(2)]

In [ ]:
# dpp.boluome.com:ctrip.tag.decomposition:
count=0
for item in c.static.jiudian_hotels.find({'cityCode':'2'},{'tag':1},no_cursor_timeout=True):
    for tag in item['tag']:
        tag_lcut=sorted(list(jieba_lcut_for_search(tag)))[:-1]
        if not tag_lcut:
            continue
        for index,lcut in enumerate(tag_lcut):
            c.srdb_new(0).zincrby(
                'dpp.boluome.com:ctrip.tag.decomposition:{}'.format(lcut),
                tag,float(index+1))
    count+=1
    print(count,end='\r')

In [ ]:
sorted(list(jieba_lcut_for_search('北京不夜谷老孟农家院')))

In [ ]:
c.static_ctrip.jiudian_hotels_30.count({}) # 2809

In [ ]:
c.static_ctrip.jiudian_hotels_30.count({'__v':10}) #2809

In [ ]:
c.static_ctrip.jiudian_hotels_30.count({'__v':10,'price':{'$exists':True},'price':{'$ne':-1}}) # 5447

In [ ]:
c.static_ctrip.jiudian_hotels_30.count({'__v':10}) # 5483

In [ ]:
c.static_ctrip.jiudian_hotels_30.update_many({},{'$rename':{'__v':'v'}})

In [ ]:
city_code_list=list(c.srdb_new(0).zrangebyscore('ctrip_city_map', 0, 6))

In [ ]:
city_code_list.remove('30')

In [ ]:
from connection import ConnectionTask
c = ConnectionTask()
from path import Path
import os
a=Path('/root/dpp/')
a.chdir()
from ctrip_worker import *
print('ok')
count=0
hotel_list = []
c.s
for city_code in c.static_cluster_ctrip.jiudian_hotels_sort.distinct("cityCode"):
#     city_code = 1236
    count+=1
    print(count,end='\r')
    for item in c.static_cluster_ctrip.jiudian_hotels_sort.find({'cityCode':city_code},{'id':1,'_id':0}):
        hotel_list.append(item['id'])
        if len(hotel_list) == 10:
            search_hotel_lowprice.apply_async(args=[city_code,hotel_ids], queue='dpp_ctrip_tasks')
            hotel_list = []
    if hotel_list:
        search_hotel_lowprice.apply_async(args=[city_code,hotel_ids], queue='dpp_ctrip_tasks')
        hotel_list = []
    break

In [ ]:
count=0
city_code_list = c.static_cluster_ctrip['jiudian_hotels_sort'].distinct("cityCode")
for city_code in city_code_list:
    for item in c.static_cluster_ctrip['jiudian_hotels_sort'].find(
        {'cityCode': city_code,'price':{'$exists':False}}, 
        {'id': 1, '_id': 0}):
        count+=1
        print(count,end='\r')
        hotel_list.append(item['id'])
        if len(hotel_list) == 20:
            resp = search_hotel_lowprice(city_code, hotel_list)
            hotel_list = []
            print(resp)
            break
        break
#             for hotel_data in get_in(resp,'HotelDataLists',[]):
#                 hotel_id = str(get_in(hotel_data, 'HotelStatusEntity.Hotel'))
#                 lowprice = get_in(hotel_data, 'HotelStatusEntity.MinPrice', default=-1)
#                 if hotel_id:
#                     print(hotel_id)
            
#     if hotel_list:
#         resp = search_hotel_lowprice(city_code, hotel_list)
#         for hotel_data in get_in(resp,'HotelDataLists',[]):
#             lowprice = get_in(hotel_data, 'HotelStatusEntity.MinPrice', default=-1)
#             if hotel_id:
#                 print(hotel_id)
#         hotel_list = []

In [ ]:
city_code,hotel_list
search_hotel_lowprice(city_code, hotel_list)

In [ ]:
search_hotel_lowprice('30', '1549637,1595181,1596231,1599182,1596049,1598159,1799001,1758471,1598808,1795280,1600185,1847462,1604027,1596942,1740590,1759734,1759984,1759887,1851047,1603097')

In [ ]:
search_hotel_lowprice('30', '1549637')

In [ ]:
# 1.4 最低价查询接口 SearchHotelLowPrice(透明代理接口)
# 功能:获取指定城市下各酒店的最低价。酒店列表页的起价常用到此接口
# 请求体示例:
def search_hotel_lowprice(city_code,hotel_list):
    if isinstance(hotel_list, list):
        hotel_list = ",".join(hotel_list)
    body={
        "SearchTypeEntity": {# <!--SearchType:查询类型;WirelessSearch，国内酒店;OnLineIntlGTASearch，海外酒店-->
            "SearchType": "WirelessSearch", # <!—分页参数，每页包含酒店数量-->
            "HotelCount": 20, # <!—页码-->
            "PageIndex": 1
        },
        "PublicSearchParameter": {
            "City": city_code,
            "CheckInDate": "2018-04-09", 
            "CheckOutDate": "2018-04-11", 
            "HotelList": hotel_list,
            "FilterRoomByPerson": "2,999",
            # <!—可选项，请求单人或多人最低价，1 代表单人价，2,999 代表 2 人及以上--> 
            "OnlyPPPrice":False, # <!—可选项，值为 true 只返回预付的最低价房型;false 则不作筛选处理 -> 
            "OnlyFGPrice":False, # <!—可选项，值为 true 只返回到付的最低价房型，false 则不作筛选出来-> },
    #         "FacilityEntity":{
    #             "IsJustifyConfirm":"T" # <!—可选项，值为 T 只返回可立即确认房型，值为 F 则不返回可立即确认房型，值为空则不筛选-->
    #         }
        }
    }
    url_html = 'http://openservice.ctrip.com/OpenService/ServiceProxy.ashx'
    token_fetch_and_update()
    Access_Token = c.dpprdb(0).hget('ctrip_token', 'access_token')
    payload = {
        'AID': conf.XIECHENG_AID,
        'SID': conf.XIECHENG_SID,
        'ICODE':'ea2570c91b3c4d19ab63fc6d97f8d62d',
        'Token': Access_Token,
        'mode': 1,
        'format': 'json'
    }
    requests_result = requests.post(url_html, params=payload, json=body)
    response = requests_result
    return response.json()

In [ ]:
count = 0
for collection in c.static_cluster_ctrip.collection_names():
    if collection in ['jiudian_hotels_sort']:
        continue
    print(collection)
    for item in c.static_cluster_ctrip[collection].find({},{'_id':0,'images':1,'id':1}):
        c.static_cluster_ctrip['jiudian_hotels_sort'].update_one({'id':item['id']},{'$set':item})
        count += 1
        print(count,end='\r')

In [ ]:
count = 0
for collection in c.static_cluster_ctrip.collection_names():
    if collection in ['jiudian_hotels_sort','jiudian_hotels_7','jiudian_hotels_578']:
        continue
    print(collection)
    for item in c.static_cluster_ctrip[collection].find({},{'_id':0,'refPoints':1,'id':1}):
        refPoints = item['refPoints']
        if not refPoints:
            continue
        if isinstance(refPoints,dict):
            refPoints = [refPoints]
        for ref_item in refPoints:
            if 'Distance' in ref_item:
                ref_item['Distance'] = float(ref_item['Distance'])
        c.static_cluster_ctrip[collection].update_one(
            {'id':item['id']},
            {'$set':{'refPoints':refPoints}}
        )
        c.static_cluster_ctrip['jiudian_hotels_sort'].update_one(
            {'id':item['id']},
            {'$set':{'refPoints':refPoints}}
        )
        count += 1
        print(count,end='\r')

In [ ]:
hotel_id_list = c.static_cluster_ctrip['jiudian_hotels_sort'].distinct('id',{'name':{'$exists':False}})


In [ ]:
for i in range(len(hotel_id_list)//10+1):
    print(i,hotel_id_list[i*10:i*10+10],end='\r')
    break
    

In [ ]:
print(hotel_id_list)

In [20]:
message = {
    'exchangeName': 'session', 
    'routingKey': 'task', 
    'payload': {
        'taskName': 'worker.ctrip.ctrip_fetch', 
        'args': {}, 
        'queue': 'ctrip_tasks'
    }
}
for item in c.pmdb.order_jiudian.find(
    {'PaymentType':'SelfPay','status':0},
    {'id':1,'appCode':1,'_id':0,'partnerId':1}
).sort([('_id',-1)]):
    item['orderId'] = item.pop('id')
    item['paymentType'] = 'SelfPay'
    message['payload']['args'] = item
    resp=requests.post('http://rmqrestapi.localdomain:8672/rabbitmq/publish',json=message)
    print(item['orderId'],resp.json())


ala204414312303658 {'code': 0, 'message': 'publish ok'}
ala204414309008782 {'code': 0, 'message': 'publish ok'}
ala204414308254970 {'code': 0, 'message': 'publish ok'}
ala204405291562019 {'code': 0, 'message': 'publish ok'}
ala204405290892013 {'code': 0, 'message': 'publish ok'}
ala204400941377772 {'code': 0, 'message': 'publish ok'}
ala204400330844070 {'code': 0, 'message': 'publish ok'}
ala204389845897522 {'code': 0, 'message': 'publish ok'}
ala204387802337376 {'code': 0, 'message': 'publish ok'}
ala204376725041472 {'code': 0, 'message': 'publish ok'}
ala204371243608769 {'code': 0, 'message': 'publish ok'}
ala204368306824970 {'code': 0, 'message': 'publish ok'}
ala204220329309549 {'code': 0, 'message': 'publish ok'}
ala204220328785260 {'code': 0, 'message': 'publish ok'}
ala204072393181072 {'code': 0, 'message': 'publish ok'}
ala204007863282519 {'code': 0, 'message': 'publish ok'}
ala204006763980776 {'code': 0, 'message': 'publish ok'}
ala204006736864967 {'code': 0, 'message': 'publi

ala201636896742366 {'code': 0, 'message': 'publish ok'}
ala201636894645213 {'code': 0, 'message': 'publish ok'}
ala201636893596636 {'code': 0, 'message': 'publish ok'}
ala201636893072347 {'code': 0, 'message': 'publish ok'}
ala201633135630354 {'code': 0, 'message': 'publish ok'}
ala201627765478062 {'code': 0, 'message': 'publish ok'}
ala201627763643053 {'code': 0, 'message': 'publish ok'}
ala201627762856618 {'code': 0, 'message': 'publish ok'}
ala201619325224901 {'code': 0, 'message': 'publish ok'}
ala201613223703645 {'code': 0, 'message': 'publish ok'}
ala201613223048284 {'code': 0, 'message': 'publish ok'}
ala201609935236036 {'code': 0, 'message': 'publish ok'}
ala201609934449603 {'code': 0, 'message': 'publish ok'}
ala201607631643918 {'code': 0, 'message': 'publish ok'}
ala201604092299839 {'code': 0, 'message': 'publish ok'}
ala201604091775550 {'code': 0, 'message': 'publish ok'}
ala201593328795803 {'code': 0, 'message': 'publish ok'}
ala201593327878298 {'code': 0, 'message': 'publi